## Purpose
- Sentiment Analysis

### Import libraries, raw data

In [1]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
ls

1a_Get_AirbnbData.ipynb
1b_quora-scrapper[inActive].ipynb
1c_Twitter-read.ipynb
2a_SentimentAnalyzer.ipynb
2b_Data_Gen.ipynb
3a_EmbeddingsAndModels.ipynb
4a_GenerateRASA_Conversational_Data copy.ipynb
5a_offlineRecommendation.ipynb
6a_NEO4J_QUERIES.docx
Architecture/
Data/
LICENSE
Papers/
Personas/
RASA/
RASA_Neo4j/
README.md
Twitter_APIKey.ipynb
models/
workspace.code-workspace


In [3]:
root='./Data/raw/'
processor = './Data/processing/'

In [4]:
reviews=pd.read_csv(root+'reviews.csv.gz')
reviews=reviews.dropna()

#### Compute rating from review : StandfordNLP Sentiment Analysis

In [831]:
# !pip install pycorenlp
# https://towardsdatascience.com/natural-language-processing-using-stanfords-corenlp-d9e64c1e1024

In [832]:
from pycorenlp import StanfordCoreNLP
# Need to start the java server.stanford-corenlp-4.3.1 % java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 50000
nlp_wrapper = StanfordCoreNLP('http://localhost:9000')

ModuleNotFoundError: No module named 'pycorenlp'

In [ ]:
def computeRatingfromReview(doc):
    overall_sentiment=0
    count=0
    
    annot_doc = nlp_wrapper.annotate(doc,
    properties={
       'annotators': 'sentiment',
       'outputFormat': 'json',
       'timeout': 100000,
    })
    
    try:
        for s in annot_doc["sentences"]:
            sentiment = s["sentiment"]
            if(sentiment=='Verypositive'):
                overall_sentiment = overall_sentiment+5
            elif(sentiment=='Positive'):
                overall_sentiment = overall_sentiment+4
            elif(sentiment=='Neutral'):
                overall_sentiment = overall_sentiment+3
            elif(sentiment=='Negative'):
                overall_sentiment = overall_sentiment+2
            elif(sentiment=='Verynegative'):
                overall_sentiment = overall_sentiment+1
            count+=1

    except:
            return 0
        
    return ((int)(overall_sentiment/count))

In [ ]:
len(reviews)

397185

In [ ]:
import time
st = time.time()
print(st)

In [ ]:
reviews['rating'] = reviews['comments']

for x in range(len(reviews.index)):
    try:
        reviews['rating'].iloc[x] = computeRatingfromReview(reviews['comments'].iloc[x])
    except:
        reviews['rating'].iloc[x] = 0

/var/folders/18/1rt0491s0d5c46mtpnlpm6000000gn/T/ipykernel_35434/906954042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['rating'] = reviews['comments']
/Users/sudhavijayakumar/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
reviews = reviews[['listing_id','reviewer_id','rating']]
reviews = reviews.rename(columns={"listing_id": "business_id", "reviewer_id": "user_id","rating":"stars"})

In [ ]:
reviews.head()

,business_id,user_id,stars
0,2818,10952,3
1,2818,12798,4
2,2818,11869,3
3,2818,14064,4
4,2818,17977,3


In [ ]:
reviews.to_csv(processor+'Processed_Airbnb/ratings_filter.csv',index=False)

In [ ]:
end = time.time()
print(end)